In [12]:
!pip install --user tweepy==3.3.0

import tweepy
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tqdm import tqdm_notebook as tqdm
import importlib
from textblob import TextBlob
import datetime as dt
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# re = python module for regular expression
import re
from datetime import datetime, timedelta
 
# # # # EVOLUTION - TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, consumer_key, consumer_secret, access_token, access_secret):
        self.auth = OAuthHandler(consumer_key, consumer_secret)
        self.auth.set_access_token(access_token, access_secret)
        api = tweepy.API(self.auth, wait_on_rate_limit=True)
        self.twitter_client = API(self.auth)
        #self.twitter_user = twitter_user
 
    def get_twitter_client_api(self):
        return self.twitter_client
 
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
 
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list
 
    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

In [13]:
# # # # TWITTER STREAM LISTENER - Analyzer # # # #
# we can use TextBlob for tweet sentiment analysis and determine the polarity of tweet, from -3 to 3;
# -3 = negative, 0 = neutral, 3 = positive
 
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    # claen_tweet using the re module to clean tweets text 
    def clean_tweet(self, tweet_text):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet_text).split())
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 3
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -3

In [17]:
query = '$AAPL'
#query = '$AAPL, $AMZN, $MSFT, $GOOGL, $FB, $BABA, $0700, $BRK, $V, $WMT, $JNJ, $TSM, $TSLA, $PG'
searched_tweets = []
tweets_repository = []
last_id = -1
days_to_subtract = 7

#if you want to select the fixed data range (7dd)
stdate = datetime.today() - timedelta(days=days_to_subtract)
endate = datetime.today()


max_tweets = 100

consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'


#consumer_key = 'A4y6ir5ElROyMEiW2WeTQlOXZ'
#consumer_secret = 'XYh3oKBJPfNTpnafqDUd4sYb2qe9yHLHHSbzGEInQ4iIlwVBtz'
#access_token = '381610804-WSQku72JaKFK7IgeI4fVhRBdpf0bP85WR9Dd3DNM'
#access_secret = 'o131rMJs1O7jFjHbvrKuRmNOPRuQCmkSsKh8Fuu9oxIu6'
 
tc = TwitterClient(consumer_key=consumer_key, 
                               consumer_secret=consumer_secret, 
                               access_token=access_token, 
                               access_secret=access_secret)
api = tc.get_twitter_client_api()
tweet_analyzer = TweetAnalyzer()
 
while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(
        q=query, 
        lang='en', 
        count=100, 
        since=stdate.strftime("%Y-%m-%d"), 
        until=endate.strftime("%Y-%m-%d"),
        include_entities=True,
        tweet_mode='extended')
        if not new_tweets:
            break
        # add the data found to the list
        searched_tweets.extend(new_tweets)
                # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

In [18]:
# Create our dict with our data
dict_ = { 'news_date': [], 'id': [], 'user': [], 'date': [], 'text': [], 'hashtags': [], 'ticker' : [], 'location': [],
         'favorite_count': [], 'statuses_count': [], 'is_quote_status': [], 
        'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in new_tweets:
    status = status_j._json
    #if len(status['entities']['symbols']) > 1:
    #   continue
    dict_['news_date'].append(status['created_at'])
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['full_text'])
    dict_['hashtags'].append(", ".join([hashtag['text'] for hashtag in status['entities']['hashtags']]))
    dict_['ticker'].append(", ".join(symbols['text'] for symbols in status['entities']['symbols'] ))
    dict_['location'].append(status['user']['location'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['statuses_count'].append(status['user']['statuses_count'])
    dict_['is_quote_status'].append(status['is_quote_status'])
    dict_['retweet'].append(status['retweeted'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['user_mentions'].append(", ".join([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']]))
    


In [19]:
#we create the dataframe pandas
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index('user')
df.sort_values(by='hashtags', inplace=True, ascending=False)

#add the new column 'sentiment' by passing the class TwitterAnalyzer

df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['text']])

#salvataggio file csv con stringa del TS convertito in data

df.to_csv(f'tweet_{endate.strftime("%Y%m%d")}' + '_' + f'{stdate.strftime("%Y%m%d")}' + '_' + f'{query.replace("$" , "")}' + '.csv', index=False)


df.head(100)

,news_date,id,date,text,hashtags,ticker,location,favorite_count,statuses_count,is_quote_status,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions,sentiment
user,,,,,,,,,,,,,,,,
mediasentiment,Thu Sep 03 23:58:51 +0000 2020,1301671027597025282,Thu Sep 03 23:58:51 +0000 2020,$AAPL new alert at https://t.co/A7qrDarJHY #...,"stocks, daytrading, NYSE, NASDAQ, market",AAPL,,0,2343328,False,False,0,6369,NaN,,3
musicnews360,Thu Sep 03 23:58:40 +0000 2020,1301670980339879937,Thu Sep 03 23:58:40 +0000 2020,$AAPL $TSLA $MSFT $SPY $AMZN $NVDA $FB $GOOG $...,"music, entertainment, musicvideo, listentothis...","AAPL, TSLA, MSFT, SPY, AMZN, NVDA, FB, GOOG, D...",🌎 Worldwide,0,115915,False,False,0,4459,NaN,,0
Samroth61562816,Thu Sep 03 23:49:50 +0000 2020,1301668756964474880,Thu Sep 03 23:49:50 +0000 2020,🇺🇸 ALAN GREENSPAN $DXY 🇺🇸\n\n$gdx $gld $slv $n...,forex,"DXY, gdx, gld, slv, nem, f, ge, aapl, msft, aa...",,0,1846,False,False,1,36,NaN,,3
Samroth61562816,Thu Sep 03 23:49:46 +0000 2020,1301668739692277760,Thu Sep 03 23:49:46 +0000 2020,🇺🇸 JAY POWELL $DXY 🇺🇸\n\n$gdx $gld $slv $nem $...,forex,"DXY, gdx, gld, slv, nem, f, ge, aapl, msft, aa...",,1,1846,False,False,1,36,NaN,,3
SeekingAlpha,Thu Sep 03 23:51:37 +0000 2020,1301669205180178432,Thu Sep 03 23:51:37 +0000 2020,$AAPL - Apple: A Lot Of Emotion And Little Com...,"finance, investing, stockmarket",AAPL,,5,750080,False,False,1,145659,NaN,,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1ksteezus,Thu Sep 03 23:54:55 +0000 2020,1301670038559105024,Thu Sep 03 23:54:55 +0000 2020,RT @scottbudman: Apple lost $180 billion in ma...,,AAPL,Bay Area,0,29143,False,False,29,703,NaN,scottbudman,0
pjl001,Thu Sep 03 23:55:00 +0000 2020,1301670060235395075,Thu Sep 03 23:55:00 +0000 2020,$AAPL $AAPL $FB - Apple on pace for biggest on...,,"AAPL, AAPL, FB","Massachusetts, USA",1,7008,False,False,0,167,NaN,,3
promotionalwit,Thu Sep 03 23:55:08 +0000 2020,1301670090539106306,Thu Sep 03 23:55:08 +0000 2020,All those people that jumped head first into $...,,AAPL,,0,7487,False,False,0,91,NaN,,3
